## Simple Request Example

Example of a simple request searching for the combination Machine AND (boolean) coronary learning in the configured metadata fields and abstracts of the articles in the database.

In [36]:
import requests

apikey = 'apikey=8m22c725rj99bvxq6pftexqk'
address = 'http://ieeexploreapi.ieee.org/api/v1/search/articles?'
querytext = '&meta_data=((Machine%20Learning)%20AND%20coronary)'
maxrecords = "&max_records=200"

url = address + apikey + querytext + maxrecords

r = requests.get(url)

print("url: \n", r.url)
print("chaves \n", r.json().keys())
print("Total Records:", r.json().get('total_records'))
print("Total de recebidos:",len(r.json().get('articles')))


url: 
 http://ieeexploreapi.ieee.org/api/v1/search/articles?apikey=8m22c725rj99bvxq6pftexqk&meta_data=((Machine%20Learning)%20AND%20coronary)&max_records=200
chaves 
 dict_keys(['total_records', 'articles', 'total_searched'])
Total Records: 64
Total de recebidos: 64


## IEEE_Xplore_Searcher class

Articles search class for IEEE Explore database using your api. 

In [25]:
import requests

class IEEE_Xplore_Searcher():
    apikey = '8m22c725rj99bvxq6pftexqk'
    address = 'http://ieeexploreapi.ieee.org/api/v1/search/articles?'
    
    def __init__(self, queryterms: list=None, apikey: str=None, address: str=None): 
        if queryterms: 
            self.queryterms = queryterms
        if apikey: 
            self.apikey = apikey
        if address: 
            self.address = address
            
    def search(self, queryterms: list=None, search_type: str="meta_data",
                    start_year: int=None, end_year: int=None,
                    content_type: str=None, start_record: int=None, 
                    sort_field: str=None, sort_order: int=None, 
                    max_records: int=200, article_title: str=None,
                    author: str=None):
        """
        @param queryterms: list of lists. Terms within the same list are
            separated by an OR. Lists are separated by an AND
        @param search_type: meta_data or querytext. 
            meta_data: This field enables a free-text search of all 
                configured metadata fields and the abstract. Accepts 
                complex queries involving field names and  boolean 
                operators.
            querytext: This field enables a free-text search of all 
                configured metadata fields, abstract and document text. 
                Accepts complex queries involving field names and boolean 
                operators. 
        @param start_year: Start value of Publication Year to restrict results by.
        @param end_year: End value of Publication Year to restrict results by.
        @param content_type: Note: these are case sensitive and must be spelled as 
            presented here to get a result: Journals, Conference, Early Access,
            Standards, Books, Courses
        @param start_record: Sequence number of first record to fetch. Default: 1
        @param sort_field: Field name on which to sort. Choose from: article_number
            article_title, author, publication_title, publication_year
        @param sort_order: asc (for ascending sort) or desc (for descending sort)
        @param max_records: The number of records to fetch. Maximum: 200
        @param article_title: Title of an individual document (journal article, 
            conference paper, standard, eBook chapter, or course).
        @param author: An author's name. Searches both first name and last name
        
        @return  the data fields returned by the search are described by the 
            following link https://developer.ieee.org/docs/read/Metadata_API_responses
        """
        
        if not queryterms: 
            queryterms = self.queryterms
        
        formated_query = "("
        for index_group, group in enumerate(queryterms):
            if index_group > 0: 
                formated_query += '%20AND%20('
    #         else: 
    # #             formated_query += '('
            for index_term, term in enumerate(group): 
                if index_term > 0: 
                    formated_query += '%20OR%20'
                if ' ' in term: 
                    formated_query += '('
                formated_query += term.replace(' ', "%20")
                if ' ' in term: 
                    formated_query += ')'

                if (index_term + 1) == len(group):
                    formated_query += ')'

        url = self.address + 'apikey=' + self.apikey  
        if search_type == "meta_data":
            url += '&meta_data=' + formated_query
        elif search_type == "querytext": 
            url += '&querytext=' + formated_query

        if start_year: 
            url += '&start_year=' + str(start_year)
        if end_year: 
            url += '&end_year=' + str(end_year)
        if content_type: 
            url += '&content_type=' + content_type
        if start_record: 
            url += '&start_record=' + start_record
        if sort_field: 
            url += '&sort_field=' + sort_field 
        if sort_order: 
            url += '&sort_order=' + sort_order
        if max_records: 
            url += '&max_records=' + str(max_records)
        if article_title: 
            url += '&article_title=' + article_title
        if author: 
            url += '&author=' + author

        return requests.get(url)


## Using the IEEE_Xplore_Searcher class

In [38]:
technology_queryterms = [
    'machine learning', 'deep learning', 'artificial intelligence', 
    'neural network', 'scoring system'
]

health_queryterms = [
    'coronary artery disease', 'chest pain', 'heart disease', 'MACE', 
    'Acute Cardiac Complications'
]

queryterms = [technology_queryterms, health_queryterms]

ieee_searcher = IEEE_Xplore_Searcher()
search = ieee_searcher.search(queryterms=queryterms,
                  start_year=1975, content_type="Journals", 
                  search_type="meta_data")  

print("url: \n", search.url)
print("\nTotal de registros:", search.json().get('total_records'))
print("Total de recebidos:",len(search.json().get('articles')))
print("\nInformações dos artigos: \n ", list(search.json().get('articles')[0].keys()))


url: 
 http://ieeexploreapi.ieee.org/api/v1/search/articles?apikey=8m22c725rj99bvxq6pftexqk&meta_data=((machine%20learning)%20OR%20(deep%20learning)%20OR%20(artificial%20intelligence)%20OR%20(neural%20network)%20OR%20(scoring%20system))%20AND%20((coronary%20artery%20disease)%20OR%20(chest%20pain)%20OR%20(heart%20disease)%20OR%20MACE%20OR%20(Acute%20Cardiac%20Complications))&start_year=1975&content_type=Journals&max_records=200

Total de registros: 118
Total de recebidos: 118

Informações dos artigos: 
  ['issn', 'start_page', 'volume', 'doi', 'citing_paper_count', 'authors', 'publication_number', 'pdf_url', 'abstract', 'title', 'abstract_url', 'publication_date', 'partnum', 'is_number', 'rank', 'article_number', 'end_page', 'index_terms', 'issue', 'publication_title', 'content_type', 'publisher', 'access_type']
